In [1]:
%pip install pandas -U -q
%pip install numpy -U -q
%pip install yfinance -U -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [27]:
from typing import List, Dict

import pandas as pd
import numpy as np
import yfinance as yf

## Unir as ações que compõem os índices IBOV e IDIV

In [3]:
def get_tickers_from_market_index(source: str):
  market_index = pd.read_csv(source)
  market_index.drop(columns="Part. (%)", inplace=True)
  market_index.dropna(inplace=True)
  return market_index.iloc[:, 0].to_list()

def merge_tickers_from_market_indexes(*args: List[str]) -> List[str]:
  result = set()

  for market_index in args:
    result = result.union(market_index)

  return list(result)

In [4]:
ibov_composition = get_tickers_from_market_index("./assets/IBOV-composition.csv")
ibov_composition

['RRRP3',
 'ALSO3',
 'ALPA4',
 'ABEV3',
 'ARZZ3',
 'ASAI3',
 'AZUL4',
 'B3SA3',
 'BBSE3',
 'BBDC3',
 'BBDC4',
 'BRAP4',
 'BBAS3',
 'BRKM5',
 'BRFS3',
 'BPAC11',
 'CRFB3',
 'BHIA3',
 'CCRO3',
 'CMIG4',
 'CIEL3',
 'COGN3',
 'CPLE6',
 'CSAN3',
 'CPFE3',
 'CMIN3',
 'CVCB3',
 'CYRE3',
 'DXCO3',
 'ELET3',
 'ELET6',
 'EMBR3',
 'ENGI11',
 'ENEV3',
 'EGIE3',
 'EQTL3',
 'EZTC3',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GOLL4',
 'NTCO3',
 'SOMA3',
 'HAPV3',
 'HYPE3',
 'IGTI11',
 'IRBR3',
 'ITSA4',
 'ITUB4',
 'JBSS3',
 'KLBN11',
 'RENT3',
 'LWSA3',
 'LREN3',
 'MGLU3',
 'MRFG3',
 'BEEF3',
 'MRVE3',
 'MULT3',
 'PCAR3',
 'PETR3',
 'PETR4',
 'RECV3',
 'PRIO3',
 'PETZ3',
 'RADL3',
 'RAIZ4',
 'RDOR3',
 'RAIL3',
 'SBSP3',
 'SANB11',
 'SMTO3',
 'CSNA3',
 'SLCE3',
 'SUZB3',
 'TAEE11',
 'VIVT3',
 'TIMS3',
 'TOTS3',
 'UGPA3',
 'USIM5',
 'VALE3',
 'VAMO3',
 'VBBR3',
 'WEGE3',
 'YDUQ3']

In [5]:
idiv_composition = get_tickers_from_market_index("./assets/IDIV-composition.csv")
idiv_composition

['ABCB4',
 'ALUP11',
 'AURE3',
 'B3SA3',
 'BRSR6',
 'BBSE3',
 'BRAP4',
 'BBAS3',
 'AGRO3',
 'CXSE3',
 'CMIG3',
 'CMIG4',
 'CSMG3',
 'CPLE3',
 'CPLE6',
 'CPFE3',
 'CMIN3',
 'DIRR3',
 'EGIE3',
 'FESA4',
 'FLRY3',
 'GGBR4',
 'GOAU4',
 'GRND3',
 'RANI3',
 'ITSA4',
 'JBSS3',
 'JHSF3',
 'KEPL3',
 'KLBN11',
 'LAVV3',
 'MRFG3',
 'LEVE3',
 'BEEF3',
 'PETR3',
 'PETR4',
 'PSSA3',
 'RAPT4',
 'ROMI3',
 'SANB11',
 'CSNA3',
 'TAEE11',
 'TASA4',
 'TGMA3',
 'VIVT3',
 'TRPL4',
 'TRIS3',
 'UNIP6',
 'USIM5',
 'VALE3',
 'WIZC3']

## Base de dados com com todos os atributos que compõem o fluxo de caixa

In [6]:
tickers = merge_tickers_from_market_indexes(ibov_composition, idiv_composition)

In [7]:
def format_tickers_to_yfinance(tickers: List[str]) -> List[str]:
  return [f"{ticker}.SA" for ticker in tickers]

tickers = format_tickers_to_yfinance(tickers)

In [8]:
cash_flow_data = pd.DataFrame()

for ticker in tickers:
    cashflow = yf.Ticker(ticker).quarterly_cashflow
    
    # Define o ticker como a coluna principal no MultiIndex
    cashflow.columns = pd.MultiIndex.from_product([[ticker], cashflow.columns])
    
    if cash_flow_data.empty:
        cash_flow_data = cashflow
        continue

    cash_flow_data = pd.concat([cash_flow_data, cashflow], axis=1)


In [22]:
cash_flow_data.fillna(0, inplace=True)
cash_flow_data

PETZ3.SA                            \
                                  2023-06-30   2023-03-31   2022-12-31   
Free Cash Flow                    14357000.0  -28149000.0   -7503000.0   
Repayment Of Debt                -24577000.0  -26781000.0  -24009000.0   
Issuance Of Debt                 200000000.0  200000000.0          0.0   
Issuance Of Capital Stock           673000.0     249000.0      56000.0   
Capital Expenditure              -54227000.0  -63974000.0 -119950000.0   
...                                      ...          ...          ...   
Sale Of Investment Properties            0.0          0.0          0.0   
Interest Paid Supplemental Data          0.0          0.0          0.0   
Net Preferred Stock Issuance             0.0          0.0          0.0   
Preferred Stock Issuance                 0.0          0.0          0.0   
Sale Of Intangibles                      0.0          0.0          0.0   

                                                 EMBR3.SA               \
                                  2022-09-30   2023-03-31   2022-12-31   
Free Cash Flow                   -38464000.0 -739700000.0  450800000.0   
Repayment Of Debt                -25889000.0 -510100000.0  -13700000.0   
Issuance Of Debt                         0.0  657600000.0   74700000.0   
Issuance Of Capital Stock                0.0          0.0          0.0   
Capital Expenditure             -103915000.0  -71000000.0  -96000000.0   
...                                      ...          ...          ...   
Sale Of Investment Properties            0.0          0.0          0.0   
Interest Paid Supplemental Data          0.0          0.0          0.0   
Net Preferred Stock Issuance             0.0          0.0          0.0   
Preferred Stock Issuance                 0.0          0.0          0.0   
Sale Of Intangibles                      0.0          0.0          0.0   

                                                  CPLE3.SA                \
                                  2022-09-30    2023-06-30    2023-03-31   
Free Cash Flow                   262100000.0  8.418920e+08  1.507660e+08   
Repayment Of Debt                -80600000.0 -1.078540e+08 -8.228800e+07   
Issuance Of Debt                  59800000.0  1.631303e+09  1.300000e+09   
Issuance Of Capital Stock                0.0  0.000000e+00  0.000000e+00   
Capital Expenditure              -61600000.0 -9.046400e+07 -2.174800e+07   
...                                      ...           ...           ...   
Sale Of Investment Properties            0.0  0.000000e+00  0.000000e+00   
Interest Paid Supplemental Data          0.0  0.000000e+00  0.000000e+00   
Net Preferred Stock Issuance             0.0  0.000000e+00  0.000000e+00   
Preferred Stock Issuance                 0.0  0.000000e+00  0.000000e+00   
Sale Of Intangibles                      0.0  0.000000e+00  0.000000e+00   

                                              ...      SUZB3.SA                \
                                  2022-12-31  ...    2022-12-31    2022-09-30   
Free Cash Flow                   189642000.0  ...  1.899817e+09  4.405940e+08   
Repayment Of Debt               -856590000.0  ... -1.144449e+09 -1.064607e+09   
Issuance Of Debt                    753000.0  ...  9.942340e+08  7.639100e+07   
Issuance Of Capital Stock                0.0  ...  0.000000e+00  0.000000e+00   
Capital Expenditure             -493586000.0  ... -4.088355e+09 -5.147783e+09   
...                                      ...  ...           ...           ...   
Sale Of Investment Properties            0.0  ...  0.000000e+00  0.000000e+00   
Interest Paid Supplemental Data          0.0  ...  0.000000e+00  0.000000e+00   
Net Preferred Stock Issuance             0.0  ...  0.000000e+00  0.000000e+00   
Preferred Stock Issuance                 0.0  ...  0.000000e+00  0.000000e+00   
Sale Of Intangibles                      0.0  ...  0.000000e+00  0.000000e+00   

                                     CSNA3.SA                      

In [23]:
cash_flow_data.to_csv("./assets/brazilian_companies_complete_cashflow.csv")

## Base de dados  com os atributos mais relevantes para o cálculo do fluxo de caixa livre

O critério de escolha dos atributos mais relevantes é feito da seguinte maneira:

1. Cálculo da correlação de todos os atributos em relação ao atributo "Free Cash Flow" para cada ativo da base de dados;
2. Obter todos os atributos classificados como correlação forte ($cor \geq 0.7$) de cada ativo;
3. Obter os 3 atributos com maior frequência com base na etapa anterior.

Considerações importantes:

- Não foi escolhido o critério da correlação média dos atributos em relação ao "Free Cash Flow", pois pelo fato da média não ser uma medida resistente (sensível a outliers). Dessa forma, isso pode afetar o resultado de algoritmos de previsão do "Free Cash Flow", uma vez que pode ocorrer de um dos atributos selecionados como "mais relevantes" não explique a variação do "Free Cash Flow" de forma apropriada.

In [24]:
tickers = list(set([ticker for ticker, _ in cash_flow_data.columns]))
tickers

['BRKM5.SA',
 'BRAP4.SA',
 'PSSA3.SA',
 'JBSS3.SA',
 'ABCB4.SA',
 'EMBR3.SA',
 'BBDC3.SA',
 'MGLU3.SA',
 'PETZ3.SA',
 'MRFG3.SA',
 'FESA4.SA',
 'PCAR3.SA',
 'SMTO3.SA',
 'CRFB3.SA',
 'WIZC3.SA',
 'EZTC3.SA',
 'PRIO3.SA',
 'GGBR4.SA',
 'PETR3.SA',
 'BRSR6.SA',
 'ROMI3.SA',
 'CSMG3.SA',
 'ASAI3.SA',
 'HYPE3.SA',
 'LAVV3.SA',
 'CPLE6.SA',
 'TRIS3.SA',
 'KLBN11.SA',
 'HAPV3.SA',
 'ALPA4.SA',
 'PETR4.SA',
 'BPAC11.SA',
 'EQTL3.SA',
 'RRRP3.SA',
 'CSAN3.SA',
 'UNIP6.SA',
 'RDOR3.SA',
 'ELET3.SA',
 'BEEF3.SA',
 'CIEL3.SA',
 'GRND3.SA',
 'GOLL4.SA',
 'CMIG4.SA',
 'USIM5.SA',
 'ABEV3.SA',
 'JHSF3.SA',
 'VBBR3.SA',
 'CYRE3.SA',
 'TIMS3.SA',
 'MULT3.SA',
 'YDUQ3.SA',
 'EGIE3.SA',
 'BRFS3.SA',
 'KEPL3.SA',
 'RAIL3.SA',
 'CSNA3.SA',
 'ALUP11.SA',
 'RAPT4.SA',
 'TASA4.SA',
 'ITSA4.SA',
 'GOAU4.SA',
 'RANI3.SA',
 'SOMA3.SA',
 'ITUB4.SA',
 'ARZZ3.SA',
 'UGPA3.SA',
 'LREN3.SA',
 'ELET6.SA',
 'CVCB3.SA',
 'B3SA3.SA',
 'IGTI11.SA',
 'AGRO3.SA',
 'WEGE3.SA',
 'RENT3.SA',
 'NTCO3.SA',
 'CMIN3.SA',
 'CMIG3.

In [26]:
correlation_results = []

for ticker in tickers:
  df = cash_flow_data[ticker].transpose()
  correlation_matrix = df.corr()["Free Cash Flow"]

  # Filtra os atributos com a correlação média >= 0.7 em relação ao atributo "Free Cash Flow"
  high_corr = correlation_matrix[correlation_matrix >= 0.7].iloc[1:]

  # Atributos com correlação forte com a coluna "Free Cash Flow"
  most_relevant = high_corr.sort_values().index.to_list()
  correlation_results.append(most_relevant)


In [37]:
import itertools
import collections


def frequency_counter(matrix: List[List[any]], n: int | None = None) -> Dict[any, int]:
    # flatten the list using chain()
    flat_list = list(itertools.chain.from_iterable(matrix))

    # count the frequencies using Counter()
    frequencies = collections.Counter(flat_list)

    if not n:
        return frequencies
    
    # Get the N most frequent attributes
    frequencies = {
        attr: frequency
        for rank, (attr, frequency) in enumerate(sorted(frequencies.items(), key=lambda item: item[1], reverse=True))
        if rank < n
    }

    return frequencies

In [38]:
frequency_counter(correlation_results)

Counter({'Operating Cash Flow': 96,
         'Change In Working Capital': 69,
         'Changes In Cash': 48,
         'End Cash Position': 33,
         'Change In Other Current Liabilities': 31,
         'Change In Payable': 30,
         'Change In Receivables': 26,
         'Investing Cash Flow': 25,
         'Capital Expenditure': 25,
         'Change In Other Current Assets': 24,
         'Repayment Of Debt': 24,
         'Cash Flow From Continuing Operating Activities': 24,
         'Net Income From Continuing Operations': 24,
         'Change In Inventory': 24,
         'Long Term Debt Payments': 23,
         'Net PPE Purchase And Sale': 22,
         'Net Issuance Payments Of Debt': 20,
         'Net Long Term Debt Issuance': 20,
         'Provisionand Write Offof Assets': 20,
         'Purchase Of Business': 19,
         'Net Other Financing Charges': 19,
         'Issuance Of Debt': 19,
         'Long Term Debt Issuance': 19,
         'Purchase Of PPE': 19,
         'Financing 

In [44]:
most_relevant = frequency_counter(correlation_results, 5)
most_relevant

{'Operating Cash Flow': 96,
 'Change In Working Capital': 69,
 'Changes In Cash': 48,
 'End Cash Position': 33,
 'Change In Other Current Liabilities': 31}

In [45]:
filter = cash_flow_data.index.isin(most_relevant)
cash_flow_data[filter]

PETZ3.SA                            \
                                      2023-06-30   2023-03-31   2022-12-31   
End Cash Position                    506721000.0  329359000.0  185411000.0   
Changes In Cash                      177362000.0  143948000.0  -77002000.0   
Operating Cash Flow                   68584000.0   35825000.0  112447000.0   
Change In Working Capital             -6126000.0  -21238000.0    9427000.0   
Change In Other Current Liabilities   19921000.0          0.0          0.0   

                                                      EMBR3.SA                \
                                      2022-09-30    2023-03-31    2022-12-31   
End Cash Position                    262413000.0  1.156400e+09  1.815600e+09   
Changes In Cash                     -129944000.0 -6.574000e+08  5.168000e+08   
Operating Cash Flow                   65451000.0 -6.687000e+08  5.468000e+08   
Change In Working Capital            -30477000.0 -5.682000e+08  3.142000e+08   
Change In Other Current Liabilities          0.0 -3.000000e+06  1.000000e+05   

                                                       CPLE3.SA                \
                                       2022-09-30    2023-06-30    2023-03-31   
End Cash Position                    1.299400e+09  4.447484e+09  2.911274e+09   
Changes In Cash                      2.599000e+08  1.536210e+09  2.328170e+08   
Operating Cash Flow                  3.237000e+08  9.323560e+08  1.725140e+08   
Change In Working Capital            3.975000e+08  4.290790e+08 -3.631090e+08   
Change In Other Current Liabilities -3.000000e+05 -6.701700e+07 -2.989400e+07   

                                                   ...      SUZB3.SA  \
                                       2022-12-31  ...    2022-12-31   
End Cash Position                    2.678457e+09  ...  9.505951e+09   
Changes In Cash                     -1.650873e+09  ...  2.746375e+09   
Operating Cash Flow                  6.832280e+08  ...  5.988172e+09   
Change In Working Capital            1.219940e+08  ... -1.578239e+09   
Change In Other Current Liabilities -6.023800e+07  ...  2.236060e+08   

                                                       CSNA3.SA                \
                                       2022-09-30    2023-06-30    2023-03-31   
End Cash Position                    6.958161e+09  1.197542e+10  1.367302e+10   
Changes In Cash                     -9.963480e+08 -1.734080e+09  1.677167e+09   
Operating Cash Flow                  5.588377e+09  1.114546e+09  5.541040e+08   
Change In Working Capital           -1.314936e+09  1.165687e+09  3.332440e+08   
Change In Other Current Liabilities -5.232200e+07  0.000000e+00  0.000000e+00   

                                                                   ROMI3.SA  \
                                       2022-12-31    2022-09-30  2023-06-30   
End Cash Position                    1.199136e+10  1.431937e+10  89609000.0   
Changes In Cash                     -2.306292e+09 -2.906329e+09 -58678000.0   
Operating Cash Flow                 -8.875400e+07  1.011298e+09  25515000.0   
Change In Working Capital           -1.188208e+09 -4.903129e+09 -30508000.0   
Change In Other Current Liabilities  0.000000e+00  0.000000e+00  23056000.0   

                                                                            
                                      2023-03-31   2022-12-31   2022-09-30  
End Cash Position                    180990000.0  131999000.0  109497000.0  
Changes In Cash                       50458000.0   20927000.0   30551000.0  
Operating Cash Flow                    6024000.0   43234000.0   30257000.0  
Change In Working Capital            -23394000.0  -61802000.0  -41928000.0  
Change In Other Current Liabilities  -13787000.0   -8983000.0   10320000.0  

[5 rows x 432 columns]

In [46]:
cash_flow_data.to_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv")